In [158]:
import json

import pandas as pd
import praw
from praw.models import MoreComments
from tqdm import tqdm

In [115]:
with open("keys.json", "r") as f:
    keys = json.load(f)

reddit_keys = keys["keys"]["reddit"]['script']
reddit = praw.Reddit(client_id=reddit_keys['client_id'],
                     client_secret=reddit_keys['secret_key'],
                     user_agent=reddit_keys['name'])

df = pd.read_csv("data/reddit_submissions/all_submissions.csv", usecols=['selftext', 'score', 'source_file', 'id']).dropna(subset=['selftext'])


In [116]:
df = df[df['selftext'] != "[deleted]"]
#df = df[df['selftext'] != "[removed]"]
df = df[df['score'] > 0]

In [117]:
fileGroup = df.groupby(["source_file"]).describe()
groups = list(fileGroup.index)
scores = [ i for i in fileGroup[('score', '50%')]]
score_thresh = list(zip(groups, scores))

In [219]:
new_thresh = []
for x in score_thresh:
    if x[0] in missing:
        new_thresh.append(x)

In [220]:
len(new_thresh)

90

In [222]:
for i in range(90):
    for x in tqdm(new_thresh):
        try:
            with open("data/reddit_comments.json", "r") as f:
                comment_forest = json.load(f)
            print(len(comment_forest.keys()))

            sub_df = df.loc[(df['source_file'].eq(x[0])) & (df['score'] >= x[1])]
            for i in sub_df['id']:
                if comment_forest.get(i):
                    continue
                submission = reddit.submission(id=i)
                submission.comments.replace_more(limit=2)
                comment_forest[i] = {}
                for comment in submission.comments.list():
                    comment_forest[i][comment.id] = {
                        "score": comment.score,
                        "text": comment.body,
                        "author": str(comment.author),
                        "created_utc": comment.created_utc
                        }
            with open("data/reddit_comments.json", "w") as fp:
                json.dump(comment_forest, fp)
        except Exception:
            with open("data/reddit_comments.json", "w") as fp:
                json.dump(comment_forest, fp)
            continue

  1%|          | 1/90 [00:28<42:15, 28.48s/it]

120145


  2%|▏         | 2/90 [01:12<48:28, 33.05s/it]

120145


  3%|▎         | 3/90 [01:51<50:42, 34.97s/it]

...

120145


 30%|███       | 27/90 [32:33<1:31:57, 87.57s/it]

120145


 30%|███       | 27/90 [1:09:54<2:43:06, 155.34s/it]


KeyboardInterrupt: 

In [214]:
ids = []
files = []
for x in score_thresh:
    print(x)
    sub_df = df.loc[(df['source_file'].eq(x[0])) & (df['score'] >= x[1])].reset_index()
    files.append(x[0])
    ids.append(sub_df['id'][0])

with open("data/reddit_comments.json", "r") as f:
        comment_forest = json.load(f)
missing = []
for idx, i in enumerate(ids):
    if not comment_forest.get(i):
        missing.append(files[idx])
    else:
        print(comment_forest[i])
    

('data/reddit_submissions/01_01_2019.json', 1.0)
('data/reddit_submissions/01_01_2020.json', 2.0)
('data/reddit_submissions/01_02_2019.json', 1.0)
('data/reddit_submissions/01_02_2020.json', 5.0)
('data/reddit_submissions/01_03_2019.json', 2.0)
('data/reddit_submissions/01_03_2020.json', 1.5)
('data/reddit_submissions/01_04_2019.json', 4.0)
('data/reddit_submissions/01_04_2020.json', 2.5)
('data/reddit_submissions/01_05_2019.json', 8.5)
('data/reddit_submissions/01_05_2020.json', 3.0)
('data/reddit_submissions/01_06_2019.json', 2.5)
('data/reddit_submissions/01_06_2020.json', 1.0)
('data/reddit_submissions/01_07_2019.json', 4.0)
('data/reddit_submissions/01_07_2020.json', 1.0)
('data/reddit_submissions/01_08_2019.json', 2.0)
('data/reddit_submissions/01_08_2020.json', 3.0)
('data/reddit_submissions/01_09_2019.json', 1.0)
('data/reddit_submissions/01_09_2020.json', 2.0)
('data/reddit_submissions/01_10_2019.json', 2.0)
('data/reddit_submissions/01_10_2020.json', 2.0)
('data/reddit_submis

('data/reddit_submissions/04_03_2020.json', 1.0)
('data/reddit_submissions/04_04_2019.json', 1.0)
('data/reddit_submissions/04_04_2020.json', 1.0)
('data/reddit_submissions/04_05_2019.json', 4.0)
('data/reddit_submissions/04_05_2020.json', 1.0)
('data/reddit_submissions/04_06_2019.json', 4.0)
('data/reddit_submissions/04_06_2020.json', 1.0)
('data/reddit_submissions/04_07_2019.json', 2.0)
('data/reddit_submissions/04_07_2020.json', 1.0)
('data/reddit_submissions/04_08_2019.json', 3.0)
('data/reddit_submissions/04_08_2020.json', 3.0)
('data/reddit_submissions/04_09_2019.json', 2.0)
('data/reddit_submissions/04_09_2020.json', 1.0)
('data/reddit_submissions/04_10_2019.json', 3.0)
('data/reddit_submissions/04_10_2020.json', 1.0)
('data/reddit_submissions/04_11_2019.json', 2.0)
('data/reddit_submissions/04_11_2020.json', 1.0)
('data/reddit_submissions/04_12_2019.json', 3.0)
('data/reddit_submissions/04_12_2020.json', 2.0)
('data/reddit_submissions/04_13_2019.json', 7.0)
('data/reddit_submis

('data/reddit_submissions/06_11_2018.json', 3.0)
('data/reddit_submissions/06_11_2019.json', 2.0)
('data/reddit_submissions/06_11_2020.json', 1.0)
('data/reddit_submissions/06_12_2018.json', 4.0)
('data/reddit_submissions/06_12_2019.json', 4.5)
('data/reddit_submissions/06_12_2020.json', 4.0)
('data/reddit_submissions/06_13_2018.json', 3.0)
('data/reddit_submissions/06_13_2019.json', 4.0)
('data/reddit_submissions/06_13_2020.json', 1.0)
('data/reddit_submissions/06_14_2018.json', 3.0)
('data/reddit_submissions/06_14_2019.json', 8.0)
('data/reddit_submissions/06_14_2020.json', 1.0)
('data/reddit_submissions/06_15_2018.json', 2.0)
('data/reddit_submissions/06_15_2019.json', 19.0)
('data/reddit_submissions/06_15_2020.json', 1.0)
('data/reddit_submissions/06_16_2018.json', 6.0)
('data/reddit_submissions/06_16_2019.json', 5.5)
('data/reddit_submissions/06_16_2020.json', 1.0)
('data/reddit_submissions/06_17_2018.json', 6.0)
('data/reddit_submissions/06_17_2019.json', 5.0)
('data/reddit_submi

('data/reddit_submissions/08_12_2018.json', 5.0)
('data/reddit_submissions/08_12_2019.json', 2.0)
('data/reddit_submissions/08_12_2020.json', 1.0)
('data/reddit_submissions/08_13_2018.json', 5.0)
('data/reddit_submissions/08_13_2019.json', 4.0)
('data/reddit_submissions/08_13_2020.json', 1.0)
('data/reddit_submissions/08_14_2018.json', 3.0)
('data/reddit_submissions/08_14_2019.json', 3.0)
('data/reddit_submissions/08_14_2020.json', 1.0)
('data/reddit_submissions/08_15_2018.json', 3.0)
('data/reddit_submissions/08_15_2019.json', 3.0)
('data/reddit_submissions/08_15_2020.json', 1.0)
('data/reddit_submissions/08_16_2018.json', 3.0)
('data/reddit_submissions/08_16_2019.json', 7.0)
('data/reddit_submissions/08_16_2020.json', 1.0)
('data/reddit_submissions/08_17_2018.json', 5.0)
('data/reddit_submissions/08_17_2019.json', 4.0)
('data/reddit_submissions/08_17_2020.json', 1.0)
('data/reddit_submissions/08_18_2018.json', 5.0)
('data/reddit_submissions/08_18_2019.json', 4.0)
('data/reddit_submis

('data/reddit_submissions/11_01_2018.json', 1.0)
('data/reddit_submissions/11_01_2019.json', 9.5)
('data/reddit_submissions/11_02_2018.json', 1.5)
('data/reddit_submissions/11_02_2019.json', 4.0)
('data/reddit_submissions/11_03_2018.json', 3.5)
('data/reddit_submissions/11_03_2019.json', 2.0)
('data/reddit_submissions/11_04_2018.json', 1.0)
('data/reddit_submissions/11_04_2019.json', 2.0)
('data/reddit_submissions/11_05_2018.json', 2.0)
('data/reddit_submissions/11_05_2019.json', 2.0)
('data/reddit_submissions/11_06_2018.json', 1.0)
('data/reddit_submissions/11_06_2019.json', 2.0)
('data/reddit_submissions/11_07_2018.json', 3.0)
('data/reddit_submissions/11_07_2019.json', 1.0)
('data/reddit_submissions/11_08_2018.json', 2.0)
('data/reddit_submissions/11_08_2019.json', 3.5)
('data/reddit_submissions/11_09_2018.json', 4.0)
('data/reddit_submissions/11_09_2019.json', 1.0)
('data/reddit_submissions/11_10_2018.json', 7.0)
('data/reddit_submissions/11_10_2019.json', 1.0)
('data/reddit_submis

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [218]:
missing

['data/reddit_submissions/01_06_2020.json',
 'data/reddit_submissions/01_07_2020.json',
 'data/reddit_submissions/01_17_2020.json',
 'data/reddit_submissions/01_19_2020.json',
 'data/reddit_submissions/01_24_2019.json',
 'data/reddit_submissions/01_25_2020.json',
 'data/reddit_submissions/01_27_2019.json',
 'data/reddit_submissions/02_02_2020.json',
 'data/reddit_submissions/02_03_2020.json',
 'data/reddit_submissions/02_04_2020.json',
 'data/reddit_submissions/02_05_2020.json',
 'data/reddit_submissions/02_08_2020.json',
 'data/reddit_submissions/02_10_2020.json',
 'data/reddit_submissions/02_12_2020.json',
 'data/reddit_submissions/02_13_2019.json',
 'data/reddit_submissions/02_13_2020.json',
 'data/reddit_submissions/02_14_2020.json',
 'data/reddit_submissions/02_17_2020.json',
 'data/reddit_submissions/02_18_2020.json',
 'data/reddit_submissions/02_21_2019.json',
 'data/reddit_submissions/02_21_2020.json',
 'data/reddit_submissions/02_22_2020.json',
 'data/reddit_submissions/02_23_

In [7]:
from prodigy.components.loaders import JSONL, JSON, CSV, TXT, Images, ImageServer
from prodigy.components.db import connect

jsonl_stream = list(JSONL("ticker_tags.jsonl"))


db = connect()                               # uses settings from prodigy.json
db.add_dataset("r01")               # add dataset
assert "r01" in db                  # check that dataset was added
db.add_examples(jsonl_stream, ["r01"])  # add examples to dataset
dataset = db.get_dataset("r01")     # retrieve a dataset
assert len(dataset) == len(jsonl_stream)     

AssertionError: 

In [8]:
len(dataset)

726

In [3]:
list(jsonl_stream)

[{'text': 'SPY|TICKER',
  'word': 'SPY',
  'sense': 'TICKER',
  'meta': {'score': 1.0, 'sense': 'TICKER'},
  'answer': 'accept',
  '_input_hash': 2039951356,
  '_task_hash': 1753895149},
 {'text': 'TSLA|TICKER',
  'word': 'TSLA',
  'sense': 'TICKER',
  'meta': {'score': 1.0, 'sense': 'TICKER'},
  'answer': 'accept',
  '_input_hash': -470315379,
  '_task_hash': -2145047467},
 {'text': 'AAPL|TICKER',
  'word': 'AAPL',
  'sense': 'TICKER',
  'meta': {'score': 1.0, 'sense': 'TICKER'},
  'answer': 'accept',
  '_input_hash': 701792636,
  '_task_hash': 1249314149},
 {'text': 'QQQ|TICKER',
  'word': 'QQQ',
  'sense': 'TICKER',
  'meta': {'score': 0.8907999992000001, 'sense': 'TICKER'},
  '_input_hash': 543238242,
  '_task_hash': -1825508871,
  '_session_id': None,
  '_view_id': 'html',
  'answer': 'accept'},
 {'text': 'NFLX|TICKER',
  'word': 'NFLX',
  'sense': 'TICKER',
  'meta': {'score': 0.8708999753000001, 'sense': 'TICKER'},
  '_input_hash': -1148905476,
  '_task_hash': -1602743311,
  '_s